# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

import holoviews as hv
hv.extension("bokeh")


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,jamestown,42.0970,-79.2353,8.77,75,20,4.63,US,1699568844
1,1,waitangi,-43.9535,-176.5597,10.46,73,97,1.34,NZ,1699569019
2,2,port-aux-francais,-49.3500,70.2167,5.18,96,100,9.05,TF,1699569020
3,3,ribeira grande,38.5167,-28.7000,20.99,94,94,10.28,PT,1699569021
4,4,edinburgh of the seven seas,-37.0676,-12.3116,12.00,67,57,3.19,SH,1699569021


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat", 
    geo = True, 
    tiles="OSM", 
    frame_width=700, 
    frame_height=500, 
    size="Humidity", 
    scale=0.5, 
    color="City", 
    hover_cols = ["Hotel Name","Country"]
    )

#Display the map
map_plot_humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_condition = ((city_data_df["Max Temp"] > 20) & (city_data_df["Max Temp"] < 27))
ideal_wind_condition = (city_data_df["Wind Speed"] < 4.4)
ideal_humid_condition = ((city_data_df["Humidity"] > 20) & (city_data_df["Humidity"] < 60))
ideal_cloud_condition = (city_data_df["Cloudiness"] < 1)

city_data_fil = city_data_df.loc[(ideal_temp_condition & ideal_wind_condition & ideal_cloud_condition ), :]

# Drop any rows with null values
city_ideal_weather_condition = city_data_fil.dropna()

# Display sample data
city_ideal_weather_condition

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
102,102,illizi,26.4833,8.4667,24.89,13,0,3.51,DZ,1699569053
159,159,curico,-34.9833,-71.2333,26.06,36,0,4.12,CL,1699569069
211,211,gloucester point,37.2540,-76.4969,24.43,67,0,3.60,US,1699569082
224,224,tekirova,36.5017,30.5272,20.51,86,0,1.80,TR,1699569086
318,318,taoudenni,22.6783,-3.9836,23.79,14,0,4.04,ML,1699568976
320,320,vila velha,-20.3297,-40.2925,24.97,76,0,3.60,BR,1699569108
356,356,howrah,22.5892,88.3103,22.00,88,0,0.00,IN,1699569117
453,453,piney green,34.7160,-77.3202,22.10,69,0,3.60,US,1699569141
470,470,vryburg,-26.9566,24.7284,21.93,20,0,2.41,ZA,1699569145
512,512,ceres,37.5949,-120.9577,20.68,37,0,2.57,US,1699569155


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal_weather_condition.copy()

hotel_df = hotel_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
102,illizi,DZ,26.4833,8.4667,13,
159,curico,CL,-34.9833,-71.2333,36,
211,gloucester point,US,37.2540,-76.4969,67,
224,tekirova,TR,36.5017,30.5272,86,
318,taoudenni,ML,22.6783,-3.9836,14,
320,vila velha,BR,-20.3297,-40.2925,76,
356,howrah,IN,22.5892,88.3103,88,
453,piney green,US,34.7160,-77.3202,69,
470,vryburg,ZA,-26.9566,24.7284,20,
512,ceres,US,37.5949,-120.9577,37,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 100000
limit = 5
apiKey = geoapify_key
params = {
    "limit":limit,
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index, "Lng"]
    latitude = hotel_df.loc[index, "Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


illizi - nearest hotel: بوناقة
curico - nearest hotel: Radisson Hotel Curicó
gloucester point - nearest hotel: Duke Of York
tekirova - nearest hotel: Beach Club Pinara
taoudenni - nearest hotel: No hotel found
vila velha - nearest hotel: Hotel Prainha
howrah - nearest hotel: Sun India Guest House
piney green - nearest hotel: Inns of the Corps
vryburg - nearest hotel: International Hotel
ceres - nearest hotel: Howard Johnson by Wyndham Modesto Ceres
greenville - nearest hotel: Quality Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
102,illizi,DZ,26.4833,8.4667,13,بوناقة
159,curico,CL,-34.9833,-71.2333,36,Radisson Hotel Curicó
211,gloucester point,US,37.2540,-76.4969,67,Duke Of York
224,tekirova,TR,36.5017,30.5272,86,Beach Club Pinara
318,taoudenni,ML,22.6783,-3.9836,14,No hotel found
320,vila velha,BR,-20.3297,-40.2925,76,Hotel Prainha
356,howrah,IN,22.5892,88.3103,88,Sun India Guest House
453,piney green,US,34.7160,-77.3202,69,Inns of the Corps
470,vryburg,ZA,-26.9566,24.7284,20,International Hotel
512,ceres,US,37.5949,-120.9577,37,Howard Johnson by Wyndham Modesto Ceres


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat", 
    geo = True, 
    tiles="OSM", 
    frame_width=700, 
    frame_height=500, 
    size="Humidity", 
    scale=0.5, 
    color="City", 
    hover_cols = ["Hotel Name","Country"]
    )

# Display the map
map_plot_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)